In [1]:
import pandas as pd
import numpy as np

## input data

In [2]:
df_saccr = pd.read_excel('input_BACVA.xlsx')
df_saccr.head()

,CUSTOMER_NUM,NETTING_SET,EAD,MATURITY
0,41747204,BIS0010,74028.399812,16.563736
1,41747204,BIS0010,17556.113430,0.893127
2,41747204,BIS0010,9469.513543,19.429423
3,41747204,BIS0010,82403.217982,12.074124
4,41747204,BIS0010,78318.052405,11.715427


## calclation flow
 1. Add columns : product EAD by Maturity
 1. Add DF defined by basel III
 1. Add columns: EAD_MATURITY * DF
 1. Add columns: SCVA

In [3]:
# add columns: EAD * Maturity
df_saccr['EAD_MAT'] = df_saccr['EAD']*df_saccr['MATURITY']

# add columns: DF defined by Basel
df_saccr['DF'] = ((1-np.exp(-0.05*df_saccr['MATURITY']))/(0.05*df_saccr['MATURITY']))

# add columns: EAD_Maturity * DF
df_saccr['EAD_MAT_DF'] = df_saccr['EAD_MAT'] * df_saccr['DF']

# add columns: SCVA
saccr_alpha = 1.4
df_saccr['SCVA'] = df_saccr['EAD_MAT_DF'] / saccr_alpha

df_saccr.head()

,CUSTOMER_NUM,NETTING_SET,EAD,MATURITY,EAD_MAT,DF,EAD_MAT_DF,SCVA
0,41747204,BIS0010,74028.399812,16.563736,1.226187e+06,0.679991,833795.722674,595568.373339
1,41747204,BIS0010,17556.113430,0.893127,1.567984e+04,0.978001,15334.888703,10953.491931
2,41747204,BIS0010,9469.513543,19.429423,1.839872e+05,0.639725,117701.173979,84072.267128
3,41747204,BIS0010,82403.217982,12.074124,9.949467e+05,0.750727,746933.438869,533523.884906
4,41747204,BIS0010,78318.052405,11.715427,9.175295e+05,0.756820,694404.989420,496003.563872


In [4]:
groupby_obj_saccr = df_saccr.groupby('NETTING_SET')
df_saccr_scva = pd.DataFrame(groupby_obj_saccr['SCVA'].sum().reset_index())
df_saccr_scva_rename = df_saccr_scva.rename(columns={'SCVA': 'SCVA_in_NettingSet'})
df_saccr_scva_rename['square_SCVA_in_NettingSet'] = df_saccr_scva_rename['SCVA_in_NettingSet']*df_saccr_scva_rename['SCVA_in_NettingSet']

In [5]:
saccr_rho = 0.5

In [6]:
first_term = pow((saccr_rho * df_saccr_scva_rename['SCVA_in_NettingSet'].sum()) ,2)
second_term = (1- pow(saccr_rho,2)) * df_saccr_scva_rename['square_SCVA_in_NettingSet'].sum()

In [7]:
BACVA = np.sqrt(first_term + second_term)
BACVA

5733425.5905402228

In [19]:
df_saccr_bacva = pd.merge(df_saccr, df_saccr_scva_rename, on='NETTING_SET')
df_saccr_bacva.head()

,CUSTOMER_NUM,NETTING_SET,EAD,MATURITY,EAD_MAT,DF,EAD_MAT_DF,SCVA,SCVA_in_NettingSet,square_SCVA_in_NettingSet
0,41747204,BIS0010,74028.399812,16.563736,1.226187e+06,0.679991,833795.722674,595568.373339,1.720122e+06,2.958818e+12
1,41747204,BIS0010,17556.113430,0.893127,1.567984e+04,0.978001,15334.888703,10953.491931,1.720122e+06,2.958818e+12
2,41747204,BIS0010,9469.513543,19.429423,1.839872e+05,0.639725,117701.173979,84072.267128,1.720122e+06,2.958818e+12
3,41747204,BIS0010,82403.217982,12.074124,9.949467e+05,0.750727,746933.438869,533523.884906,1.720122e+06,2.958818e+12
4,41747204,BIS0010,78318.052405,11.715427,9.175295e+05,0.756820,694404.989420,496003.563872,1.720122e+06,2.958818e+12


In [13]:
df_saccr.head()

,CUSTOMER_NUM,NETTING_SET,EAD,MATURITY,EAD_MAT,DF,EAD_MAT_DF,SCVA
0,41747204,BIS0010,74028.399812,16.563736,1.226187e+06,0.679991,833795.722674,595568.373339
1,41747204,BIS0010,17556.113430,0.893127,1.567984e+04,0.978001,15334.888703,10953.491931
2,41747204,BIS0010,9469.513543,19.429423,1.839872e+05,0.639725,117701.173979,84072.267128
3,41747204,BIS0010,82403.217982,12.074124,9.949467e+05,0.750727,746933.438869,533523.884906
4,41747204,BIS0010,78318.052405,11.715427,9.175295e+05,0.756820,694404.989420,496003.563872


In [14]:
df_saccr_scva_rename.head()

,NETTING_SET,SCVA_in_NettingSet,square_SCVA_in_NettingSet
0,BIS0010,1.720122e+06,2.958818e+12
1,BIS0020,7.981512e+05,6.370453e+11
2,BIS0981,8.871280e+04,7.869961e+09
3,BIS0987,1.296386e+06,1.680615e+12
4,BIS1097,2.737743e+05,7.495237e+10


In [11]:
df_saccr.index.is_unique

True

In [73]:
'''
df_saccr_ead_by_mat = df_input_saccr.assign(EAD_MATURITY=df_input_saccr['EAD']*df_input_saccr['MATURITY'])
df_saccr_ead_by_mat.head()
df_saccr_DF = df_saccr_ead_by_mat.assign(DF=((1-np.exp(-0.05*saccr_ead_by_mat['MATURITY']))/(0.05*df_saccr_ead_by_mat['MATURITY'])))
df_input_saccr_3.head()
df_input_saccr_4  = df_input_saccr_3.assign(DF_EAD_MAT=df_input_saccr_3['EAD_MATURITY']*df_input_saccr_3['DF'])
df_input_saccr_4.head()
alpha = 1.4
df_input_saccr_5 = df_input_saccr_4.assign(SCVA = df_input_saccr_4['DF_EAD_MAT']/alpha)
df_input_saccr_5
'''

"\ndf_saccr_ead_by_mat = df_input_saccr.assign(EAD_MATURITY=df_input_saccr['EAD']*df_input_saccr['MATURITY'])\ndf_saccr_ead_by_mat.head()\ndf_saccr_DF = df_saccr_ead_by_mat.assign(DF=((1-np.exp(-0.05*saccr_ead_by_mat['MATURITY']))/(0.05*df_saccr_ead_by_mat['MATURITY'])))\ndf_input_saccr_3.head()\ndf_input_saccr_4  = df_input_saccr_3.assign(DF_EAD_MAT=df_input_saccr_3['EAD_MATURITY']*df_input_saccr_3['DF'])\ndf_input_saccr_4.head()\nalpha = 1.4\ndf_input_saccr_5 = df_input_saccr_4.assign(SCVA = df_input_saccr_4['DF_EAD_MAT']/alpha)\ndf_input_saccr_5\n"

In [19]:
df1 = pd.DataFrame({'key' : ['b','b','a','c','a','a','b'],
                                      'data1' : range(7)})
df2 = pd.DataFrame({'key' : ['a', 'b', 'c'],
                                      'data2' : range(3)})

In [21]:
pd.merge(df1,df2, on='key')

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0
6,3,c,2
